# Andreas sitt workplace, GTFO!!

knowns and unknowns

In [273]:

################################################################    main del    ####################################################################################################
# Import necessary classes from the 'classes' module
from classes import *
import pandas as pd
import os

# Get the absolute path to the current directory
pathToFile = os.path.abspath(os.curdir)

# Read network configuration data from an Excel file
df_line_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Line_data")
df_bus_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Bus_data")

#############################################################    slutt main del  ##############################################################################################


a_bus_data = df_bus_data.values

df_overview = pd.DataFrame(columns=['Bus number', 'Type', 'Known_1', 'Known_2', 'Unknown_1', 'Unknown_2'], index=[df_bus_data.index] , dtype = object)

for i in df_bus_data.index:
    df_overview.loc[i, 'Bus number'] = df_bus_data.loc[i, 'Bus Code']

    try:
        generation_mw = (df_bus_data.loc[i, 'Generation (MW)'])
        load_mw = (df_bus_data.loc[i, 'Load (MW)'])
        generation_mvar = (df_bus_data.loc[i, 'Generation (MVAr)'])
        load_mvar = (df_bus_data.loc[i, 'Load (MVAr)'])
        bus_voltage = (df_bus_data.loc[i, 'Assumed bus voltage (pu)'])
    except ValueError:
        # Handle non-integer values
        generation_mw = "-"
        load_mw = "-"
        generation_mvar = "-"
        load_mvar = "-"
        bus_voltage = "-"


    if generation_mw != "-" and load_mw != "-" and generation_mvar != "-" and load_mvar != "-":
        df_overview.loc[i, 'Type'] = 'P' + 'Q'
        df_overview.loc[i, 'Known_1'] = 'P'
        df_overview.loc[i, 'Known_2'] = 'Q'
        df_overview.loc[i, 'Unknown_1'] = 'V'
        df_overview.loc[i, 'Unknown_2'] = 'θ'
    
    elif bus_voltage != "-" and (generation_mw != "-" and load_mw != "-"):
        df_overview.loc[i, 'Type'] = 'P' + 'V'
        df_overview.loc[i, 'Known_1'] = 'P'
        df_overview.loc[i, 'Known_2'] = 'V'
        df_overview.loc[i, 'Unknown_1'] = 'Q' 
        df_overview.loc[i, 'Unknown_2'] = 'θ'

    else:
        df_overview.loc[i, 'Type'] = "Slack"
        df_overview.loc[i, 'Known_1'] = 'V'
        df_overview.loc[i, 'Known_2'] = 'θ'
        df_overview.loc[i, 'Unknown_1'] = 'P'
        df_overview.loc[i, 'Unknown_2'] = 'Q'



data_dict_P = {}

for i in df_bus_data.index:

    while(df_bus_data.loc[i, 'Load (MW)']) == '-' or (df_bus_data.loc[i, 'Generation (MW)']) == '-' :
        i+=1

    # Create dynamic variable names
    new_name_P = f"P_{i+1}"
    # Example value for the variable
    valueP = float(df_bus_data.loc[i, 'Load (MW)']) - float(df_bus_data.loc[i, 'Generation (MW)'])
    # Store the value in the dictionary with the dynamically created name
    data_dict_P[new_name_P] = valueP

data_dict_Q = {}

for i in df_bus_data.index:

    while (df_bus_data.loc[i, 'Load (MVAr)']) == '-' or (df_bus_data.loc[i, 'Generation (MVAr)']) == '-':
        i+=1
        
    # Create dynamic variable names
    new_name_Q = f"Q_{i+1}"
    # Example value for the variable
    valueQ = float(df_bus_data.loc[i, 'Load (MVAr)']) - float(df_bus_data.loc[i, 'Generation (MVAr)'])
    # Store the value in the dictionary with the dynamically created name
    data_dict_Q[new_name_Q] = valueQ 


df_known_P  = pd.DataFrame.from_dict(data_dict_P, orient='index')
df_known_Q  = pd.DataFrame.from_dict(data_dict_Q, orient='index')
df_known = pd.concat([df_known_P, df_known_Q], axis = 0)
df_known = df_known.rename(columns = {0 : 'Knowns'})

pu_base = 100
df_known_pu = df_known / pu_base


# guess values#

guess_data_dict_V = {}

for j in df_bus_data.index:

    try: 

        unkown1 = (df_overview.loc[j, 'Unknown_1']).item()
    except:
        unkown1 = '-'

    if unkown1 == 'V':
        # Create dynamic variable names
        new_name_unknown_V = f"V_{j+1}"
        # Example value for the variable
        guessV = float(df_bus_data.loc[j, 'Assumed bus voltage (pu)']) 
        # Store the value in the dictionary with the dynamically created name
        guess_data_dict_V[new_name_unknown_V] = guessV 

    else:
        j += 1



guess_data_dict_θ = {}

for j in df_bus_data.index:
    try: 

        unkown2 = (df_overview.loc[j, 'Unknown_2']).item()
    except:
        unkown2 = '-'
    

    if unkown2 == 'θ':

        # Create dynamic variable names
        new_name_unknown_θ = f"θ_{j+1}"
        # Example value for the variable
        guessθ = float(df_bus_data.loc[j, 'Angle']) 
        # Store the value in the dictionary with the dynamically created name
        guess_data_dict_θ[new_name_unknown_θ] = guessθ

    else:
        j += 1

df_guess_V  = pd.DataFrame.from_dict(guess_data_dict_V, orient='index')
df_guess_θ  = pd.DataFrame.from_dict(guess_data_dict_θ, orient='index')
df_guess_θV = pd.concat([df_guess_θ, df_guess_V], axis = 0)
df_guess_θV = df_guess_θV.rename(columns = {0 : 'Guesses'})

# Display the updated df_overview
display(df_bus_data)
display(df_overview)
display(df_known_pu)
display(df_guess_θV)



,Bus Code,Assumed bus voltage (pu),Angle,Generation (MW),Generation (MVAr),Load (MW),Load (MVAr)
0,1,1.06,0,-,-,0,0
1,2,1.00,0,40,-,20,10
2,3,1.00,0,0,0,45,15
3,4,1.00,0,0,0,40,5
4,5,1.00,0,0,0,60,10


,Bus number,Type,Known_1,Known_2,Unknown_1,Unknown_2
0,1,Slack,V,θ,P,Q
1,2,PV,P,V,Q,θ
2,3,PQ,P,Q,V,θ
3,4,PQ,P,Q,V,θ
4,5,PQ,P,Q,V,θ


,Knowns
P_2,-0.20
P_3,0.45
P_4,0.40
P_5,0.60
Q_3,0.15
Q_4,0.05
Q_5,0.10


,Guesses
θ_2,0.0
θ_3,0.0
θ_4,0.0
θ_5,0.0
V_3,1.0
V_4,1.0
V_5,1.0
